In [11]:
import numpy as np
import torch

def get_kernel_matrix(lambdas, beta):
    # (p, 1)
    lambdas = lambdas[:, None]
    # (p, p)
    K = torch.exp(-beta**2 * (lambdas - lambdas.T) ** 2)
    return K

In [38]:
device = 'cuda:0'
corrector = False
steps = 10
lambdas = torch.tensor(np.linspace(-5.0778, 5.7618, steps), device=device)

for i in range(steps):
    s = 5
    p = min(3, i+1)
    beta = steps / (np.exp(s) * abs(lambdas[-1] - lambdas[0]))
    lambda_array = torch.flip(lambdas[i-p+1:i+(2 if corrector else 1)], dims=[0]).to(torch.float64)

    kernel = get_kernel_matrix(lambda_array, beta)
    eye = torch.eye(p+1, device=kernel.device).to(torch.float64)
    kernel_aug = 1 - eye
    kernel_aug[:p, :p] = kernel
    kernel_inv = torch.linalg.pinv(kernel_aug)
    diff = torch.mean(abs(torch.eye(p+1, device=kernel.device) - kernel_aug @ kernel_inv))
    print(diff)

tensor(7.5138e-17, device='cuda:0', dtype=torch.float64)
tensor(1.0065e-13, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)
tensor(1.8577e-09, device='cuda:0', dtype=torch.float64)


In [40]:
import sympy as sp

# 1. 심볼 정의
beta = sp.Symbol('beta', positive=True)
# λ_i 들도 각각 심볼로 정의하려면 차원(dimension)을 잡아야 할 수 있는데,
# 우선 간단히 p개 지점을 각각 심볼 lambda_1, lambda_2, ... 로 정의한다고 가정.
p = 2  # 예시, 실제로는 p를 원하는 크기로 설정
lambdas = [sp.Symbol(f'lambda_{k}', real=True) for k in range(p)]

# 2. K행렬(또는 R행렬)에 들어갈 각 원소를 심볼릭으로 구성
#    여기서는 예시로 1차원상에서 |lambda_i - lambda_j|^2 로 처리
#    만약 여러 차원이면 norm(·) 함수를 적절히 정의해야 함.
K_block = []
for i in range(p):
    row = []
    for j in range(p):
        dist_sq = (lambdas[i] - lambdas[j])**2
        row.append(sp.exp(-beta**2 * dist_sq))
    K_block.append(row)

# 3. 마지막 행과 열 붙이기
#    블록 R = p×p (위에서 만든 K_block)
#    마지막 열은 [1, 1, ..., 1]^T, 마지막 행은 [1, 1, ..., 1, 0].
K = []
for i in range(p):
    K.append(K_block[i] + [sp.Integer(1)])
K.append([sp.Integer(1)]*p + [sp.Integer(0)])

# 4. Sympy Matrix 객체로 만든 후, 역행렬 계산
K_mat = sp.Matrix(K)
K_inv = K_mat.inv()  # 기호적(symbolic) 역행렬

print(K_inv)


Matrix([[-1/(2*exp(-beta**2*lambda_0**2 + 2*beta**2*lambda_0*lambda_1 - beta**2*lambda_1**2) - 2), 1/(2*exp(-beta**2*lambda_0**2 + 2*beta**2*lambda_0*lambda_1 - beta**2*lambda_1**2) - 2), 1/2], [1/(2*exp(-beta**2*lambda_0**2 + 2*beta**2*lambda_0*lambda_1 - beta**2*lambda_1**2) - 2), -1/(2*exp(-beta**2*lambda_0**2 + 2*beta**2*lambda_0*lambda_1 - beta**2*lambda_1**2) - 2), 1/2], [1/2, 1/2, (1 - exp(-2*beta**2*lambda_0**2 + 4*beta**2*lambda_0*lambda_1 - 2*beta**2*lambda_1**2))/(2*exp(-beta**2*lambda_0**2 + 2*beta**2*lambda_0*lambda_1 - beta**2*lambda_1**2) - 2)]])
